In [1]:
import sys
from pathlib import Path
nb_path = Path.cwd().parent.parent 
sys.path.insert(0, str(nb_path))

import pandas as pd

from processing.gcp.storage import gcp

pd.set_option('display.max_columns', None)

## Premier League

We will use the Premier League as the initial example to work out how to complete the join between stats, wages, and valuations properly before going on to join the remaining four leagues

Load wages and values

In [2]:
prem_df = gcp.read_df_from_bucket(
    bucket_name='joined_wages_values', blob_name='premier_league_wages_values.csv'
)

[04/01/24 14:06:18] INFO     processing.gcp.storage - [ MainThread:get_gcp_bucket:28 ] - Getting GCP bucket                              ]8;id=926134;file:///Users/conal/Projects/fpl_app/feature-processing/processing/gcp/storage.py\storage.py]8;;\:]8;id=143114;file:///Users/conal/Projects/fpl_app/feature-processing/processing/gcp/storage.py#28\28]8;;\

                    INFO     processing.gcp.storage - [ MainThread:get_gcp_bucket:33 ] - GCP bucket retrieved                            ]8;id=127722;file:///Users/conal/Projects/fpl_app/feature-processing/processing/gcp/storage.py\storage.py]8;;\:]8;id=824682;file:///Users/conal/Projects/fpl_app/feature-processing/processing/gcp/storage.py#33\33]8;;\

In [3]:
prem_df.head()

,player,squad,age,weekly_wages_euros,annual_wages_euros,season,age_range,continent,player_id,position,country,height,foot,signed_from,signing_fee_euro_mill,market_value_euro_mill,league,signed_year
0,Alexis Sánchez,Arsenal,28,402897,20950637,2017,25-29,South America,2111.0,Centre-Forward,Chile,169.0,right,FC Barcelona,42.5,70.0,premier_league,2014.0
1,Mesut Özil,Arsenal,28,402897,20950637,2017,25-29,Europe,1796.0,Attacking-Midfield,Germany,180.0,left,Real Madrid,47.0,50.0,premier_league,2013.0
2,Eden Hazard,Chelsea,26,256548,13340491,2017,25-29,Europe,1003.0,Left-Winger,Belgium,175.0,right,LOSC Lille,35.0,110.0,premier_league,2012.0
3,Cesc Fàbregas,Chelsea,30,236735,12310226,2017,30-34,Europe,744.0,Central-Midfield,Spain,179.0,right,FC Barcelona,33.0,30.0,premier_league,2014.0
4,Henrikh Mkhitaryan,Arsenal,28,230227,11971793,2017,25-29,Asia,1608.0,Central-Midfield,Armenia,177.0,both,Manchester United,34.0,35.0,premier_league,2018.0


Load standard stats

We can create a basic data set that comprises the standard performance stats and player valuations and wages. Later on we can create a dataset that takes into account more detailed stats including possession, shooting, and defending.

In [24]:
# load in stats
standard_df = gcp.read_df_from_bucket(
    bucket_name='processed_fbref_db', blob_name='processed_standard.csv'
)

[04/01/24 14:14:08] INFO     processing.gcp.storage - [ MainThread:get_gcp_bucket:28 ] - Getting GCP bucket                              ]8;id=529312;file:///Users/conal/Projects/fpl_app/feature-processing/processing/gcp/storage.py\storage.py]8;;\:]8;id=90702;file:///Users/conal/Projects/fpl_app/feature-processing/processing/gcp/storage.py#28\28]8;;\

                    INFO     processing.gcp.storage - [ MainThread:get_gcp_bucket:33 ] - GCP bucket retrieved                            ]8;id=105;file:///Users/conal/Projects/fpl_app/feature-processing/processing/gcp/storage.py\storage.py]8;;\:]8;id=470505;file:///Users/conal/Projects/fpl_app/feature-processing/processing/gcp/storage.py#33\33]8;;\

In [51]:
standard_df.head(2)

,rk,player,nation,pos,squad,comp,age,born,mp,starts,min,90s,goals,assists,goals_assists,non_penalty_goals,penalty_kicks,penalty_kicks_attempted,yellow_cards,red_cards,xg,non_penalty_xg,xag,non_pen_xg_plus_xag,progressive_carries,progressive_passes,progressive_passes_received,goals_per_90,assists_per_90,goals_assists_per_90,non_penalty_goals_per_90,non_penalty_goals_assists_per_90,xg_per_90,xag_per_90,xg_plus_xag_per_90,non_penalty_xg_per_90,non_penalty_xg_plus_xag_per_90,season,general_pos,age_range,country,continent,player_id
0,1,Patrick van Aanholt,NED,DF,Crystal Palace,Premier League,26.0,1990.0,28,25,2184,24.3,5,1,6,5,0,0,7,0,3.1,3.1,2.1,5.2,46.0,92.0,86.0,0.21,0.04,0.25,0.21,0.25,0.13,0.09,0.21,0.13,0.21,2017,Defender,25-29,Netherlands,Europe,1
1,2,Rolando Aarons,ENG,"MF,FW",Newcastle Utd,Premier League,21.0,1995.0,4,1,139,1.5,0,0,0,0,0,0,0,0,0.1,0.1,0.0,0.1,7.0,3.0,4.0,0.00,0.00,0.00,0.00,0.00,0.04,0.00,0.04,0.04,0.04,2017,Midfielder,20-24,England,Europe,2


In [54]:
# stats include all leagues
standard_df['comp'].unique()

array(['Premier League', 'Serie A', 'Ligue 1', 'La Liga', 'Bundesliga'],
      dtype=object)

In [26]:
# season columns are slightly different - change the stats df to match the wages df
standard_df.loc[:, 'season'] = standard_df['season'].str[:4]

In [30]:
standard_df.loc[:, 'season'] = standard_df['season'].astype(int)

In [31]:
standard_df['season'].dtype

dtype('O')

In [48]:
def clean_season_col(df: pd.DataFrame) -> pd.DataFrame:
    df.loc[:, 'season'] = df['season'].str[:4]
    df.loc[:, 'season'] = df['season'].astype(int)
    return df

In [42]:
# merge the two dataframes
joined_df = pd.merge(
    prem_df,
    standard_df,
    how="inner",
    left_on=["player", "season", 'squad'],
    right_on=["player", "season", 'squad'],
    suffixes=("", "_stats"),
)

In [45]:
cols_to_keep = [col for col in joined_df.columns if '_stats' not in col]
joined_df = joined_df[cols_to_keep]

In [50]:
joined_df.head(2)

,player,squad,age,weekly_wages_euros,annual_wages_euros,season,age_range,continent,player_id,position,country,height,foot,signed_from,signing_fee_euro_mill,market_value_euro_mill,league,signed_year,rk,nation,pos,comp,born,mp,starts,min,90s,goals,assists,goals_assists,non_penalty_goals,penalty_kicks,penalty_kicks_attempted,yellow_cards,red_cards,xg,non_penalty_xg,xag,non_pen_xg_plus_xag,progressive_carries,progressive_passes,progressive_passes_received,goals_per_90,assists_per_90,goals_assists_per_90,non_penalty_goals_per_90,non_penalty_goals_assists_per_90,xg_per_90,xag_per_90,xg_plus_xag_per_90,non_penalty_xg_per_90,non_penalty_xg_plus_xag_per_90,general_pos
0,Alexis Sánchez,Arsenal,28,402897,20950637,2017,25-29,South America,2111.0,Centre-Forward,Chile,169.0,right,FC Barcelona,42.5,70.0,premier_league,2014.0,2214,CHI,"MF,FW",Premier League,1988.0,19,17,1503,16.7,7,3,10,6,1,1,4,0,8.1,7.3,4.9,12.2,50.0,149.0,218.0,0.42,0.18,0.6,0.36,0.54,0.49,0.29,0.78,0.44,0.73,Midfielder
1,Mesut Özil,Arsenal,28,402897,20950637,2017,25-29,Europe,1796.0,Attacking-Midfield,Germany,180.0,left,Real Madrid,47.0,50.0,premier_league,2013.0,1881,GER,"MF,FW",Premier League,1988.0,26,24,2163,24.0,4,8,12,4,0,0,4,0,4.4,4.4,8.7,13.1,136.0,277.0,267.0,0.17,0.33,0.5,0.17,0.50,0.18,0.36,0.55,0.18,0.55,Midfielder


In [49]:
# check join has worked
joined_df.loc[joined_df['player'] == 'Kevin De Bruyne'].head(2)

,player,squad,age,weekly_wages_euros,annual_wages_euros,season,age_range,continent,player_id,position,country,height,foot,signed_from,signing_fee_euro_mill,market_value_euro_mill,league,signed_year,rk,nation,pos,comp,born,mp,starts,min,90s,goals,assists,goals_assists,non_penalty_goals,penalty_kicks,penalty_kicks_attempted,yellow_cards,red_cards,xg,non_penalty_xg,xag,non_pen_xg_plus_xag,progressive_carries,progressive_passes,progressive_passes_received,goals_per_90,assists_per_90,goals_assists_per_90,non_penalty_goals_per_90,non_penalty_goals_assists_per_90,xg_per_90,xag_per_90,xg_plus_xag_per_90,non_penalty_xg_per_90,non_penalty_xg_plus_xag_per_90,general_pos
14,Kevin De Bruyne,Manchester City,26,201448,10475318,2017,25-29,Europe,591.0,Attacking-Midfield,Belgium,181.0,right,VfL Wolfsburg,76.0,150.0,premier_league,2015.0,619,BEL,MF,Premier League,1991.0,37,36,3073,34.1,8,16,24,8,0,0,2,0,5.1,5.1,11.1,16.2,183.0,388.0,259.0,0.23,0.47,0.70,0.23,0.70,0.15,0.32,0.48,0.15,0.48,Midfielder
448,Kevin De Bruyne,Manchester City,27,267923,13931996,2018,25-29,Europe,591.0,Attacking-Midfield,Belgium,181.0,right,VfL Wolfsburg,76.0,130.0,premier_league,2015.0,588,BEL,MF,Premier League,1991.0,19,11,975,10.8,2,2,4,2,0,0,2,0,1.4,1.4,5.7,7.0,50.0,109.0,88.0,0.18,0.18,0.37,0.18,0.37,0.13,0.52,0.65,0.13,0.65,Midfielder


## Join Other 4 Leagues

Merge the stats, values, and wages data for the other 4 leagues

load in the data from each league and concat together

In [55]:
def load_data(league: str) -> pd.DataFrame:
    return gcp.read_df_from_bucket(
        bucket_name="joined_wages_values", blob_name=f"{league}_wages_values.csv"
    )

In [65]:
leagues = ['premier_league', 'bundesliga', 'la_liga', 'serie_a', 'ligue_1']

In [66]:
dfs = [load_data(league) for league in leagues]

[04/01/24 15:04:21] INFO     processing.gcp.storage - [ MainThread:get_gcp_bucket:28 ] - Getting GCP bucket                              ]8;id=490941;file:///Users/conal/Projects/fpl_app/feature-processing/processing/gcp/storage.py\storage.py]8;;\:]8;id=983391;file:///Users/conal/Projects/fpl_app/feature-processing/processing/gcp/storage.py#28\28]8;;\

[04/01/24 15:04:22] INFO     processing.gcp.storage - [ MainThread:get_gcp_bucket:33 ] - GCP bucket retrieved                            ]8;id=966664;file:///Users/conal/Projects/fpl_app/feature-processing/processing/gcp/storage.py\storage.py]8;;\:]8;id=166916;file:///Users/conal/Projects/fpl_app/feature-processing/processing/gcp/storage.py#33\33]8;;\

[04/01/24 15:04:23] INFO     processing.gcp.storage - [ MainThread:get_gcp_bucket:28 ] - Getting GCP bucket                              ]8;id=107535;file:///Users/conal/Projects/fpl_app/feature-processing/processing/gcp/storage.py\storage.py]8;;\:]8;id=36341;file:///Users/conal/Projects/fpl_app/feature-processing/processing/gcp/storage.py#28\28]8;;\

                    INFO     processing.gcp.storage - [ MainThread:get_gcp_bucket:33 ] - GCP bucket retrieved                            ]8;id=305620;file:///Users/conal/Projects/fpl_app/feature-processing/processing/gcp/storage.py\storage.py]8;;\:]8;id=620866;file:///Users/conal/Projects/fpl_app/feature-processing/processing/gcp/storage.py#33\33]8;;\

[04/01/24 15:04:25] INFO     processing.gcp.storage - [ MainThread:get_gcp_bucket:28 ] - Getting GCP bucket                              ]8;id=62191;file:///Users/conal/Projects/fpl_app/feature-processing/processing/gcp/storage.py\storage.py]8;;\:]8;id=333017;file:///Users/conal/Projects/fpl_app/feature-processing/processing/gcp/storage.py#28\28]8;;\

                    INFO     processing.gcp.storage - [ MainThread:get_gcp_bucket:33 ] - GCP bucket retrieved                            ]8;id=546300;file:///Users/conal/Projects/fpl_app/feature-processing/processing/gcp/storage.py\storage.py]8;;\:]8;id=701229;file:///Users/conal/Projects/fpl_app/feature-processing/processing/gcp/storage.py#33\33]8;;\

[04/01/24 15:04:26] INFO     processing.gcp.storage - [ MainThread:get_gcp_bucket:28 ] - Getting GCP bucket                              ]8;id=713801;file:///Users/conal/Projects/fpl_app/feature-processing/processing/gcp/storage.py\storage.py]8;;\:]8;id=970488;file:///Users/conal/Projects/fpl_app/feature-processing/processing/gcp/storage.py#28\28]8;;\

[04/01/24 15:04:27] INFO     processing.gcp.storage - [ MainThread:get_gcp_bucket:33 ] - GCP bucket retrieved                            ]8;id=752221;file:///Users/conal/Projects/fpl_app/feature-processing/processing/gcp/storage.py\storage.py]8;;\:]8;id=502751;file:///Users/conal/Projects/fpl_app/feature-processing/processing/gcp/storage.py#33\33]8;;\

                    INFO     processing.gcp.storage - [ MainThread:get_gcp_bucket:28 ] - Getting GCP bucket                              ]8;id=333827;file:///Users/conal/Projects/fpl_app/feature-processing/processing/gcp/storage.py\storage.py]8;;\:]8;id=76387;file:///Users/conal/Projects/fpl_app/feature-processing/processing/gcp/storage.py#28\28]8;;\

[04/01/24 15:04:28] INFO     processing.gcp.storage - [ MainThread:get_gcp_bucket:33 ] - GCP bucket retrieved                            ]8;id=130015;file:///Users/conal/Projects/fpl_app/feature-processing/processing/gcp/storage.py\storage.py]8;;\:]8;id=991414;file:///Users/conal/Projects/fpl_app/feature-processing/processing/gcp/storage.py#33\33]8;;\

In [67]:
# concat dataframes
all_leagues = pd.concat(dfs)

In [68]:
# join with stats
joined_all = pd.merge(
    all_leagues,
    standard_df,
    how="inner",
    left_on=["player", "season", 'squad'],
    right_on=["player", "season", 'squad'],
    suffixes=("", "_stats"),
)

In [69]:
joined_all.head(2)

,player,squad,age,weekly_wages_euros,annual_wages_euros,season,age_range,continent,player_id,position,country,height,foot,signed_from,signing_fee_euro_mill,market_value_euro_mill,league,signed_year,rk,nation,pos,comp,age_stats,born,mp,starts,min,90s,goals,assists,goals_assists,non_penalty_goals,penalty_kicks,penalty_kicks_attempted,yellow_cards,red_cards,xg,non_penalty_xg,xag,non_pen_xg_plus_xag,progressive_carries,progressive_passes,progressive_passes_received,goals_per_90,assists_per_90,goals_assists_per_90,non_penalty_goals_per_90,non_penalty_goals_assists_per_90,xg_per_90,xag_per_90,xg_plus_xag_per_90,non_penalty_xg_per_90,non_penalty_xg_plus_xag_per_90,general_pos,age_range_stats,country_stats,continent_stats,player_id_stats
0,Alexis Sánchez,Arsenal,28,402897,20950637,2017,25-29,South America,2111.0,Centre-Forward,Chile,169.0,right,FC Barcelona,42.5,70.0,premier_league,2014.0,2214,CHI,"MF,FW",Premier League,28.0,1988.0,19,17,1503,16.7,7,3,10,6,1,1,4,0,8.1,7.3,4.9,12.2,50.0,149.0,218.0,0.42,0.18,0.6,0.36,0.54,0.49,0.29,0.78,0.44,0.73,Midfielder,25-29,Chile,South America,2111
1,Mesut Özil,Arsenal,28,402897,20950637,2017,25-29,Europe,1796.0,Attacking-Midfield,Germany,180.0,left,Real Madrid,47.0,50.0,premier_league,2013.0,1881,GER,"MF,FW",Premier League,28.0,1988.0,26,24,2163,24.0,4,8,12,4,0,0,4,0,4.4,4.4,8.7,13.1,136.0,277.0,267.0,0.17,0.33,0.5,0.17,0.50,0.18,0.36,0.55,0.18,0.55,Midfielder,25-29,Germany,Europe,1796


In [70]:
joined_all.shape

(13677, 58)

In [71]:
joined_all.loc[joined_all['player'] == 'Robert Lewandowski'].head()

,player,squad,age,weekly_wages_euros,annual_wages_euros,season,age_range,continent,player_id,position,country,height,foot,signed_from,signing_fee_euro_mill,market_value_euro_mill,league,signed_year,rk,nation,pos,comp,age_stats,born,mp,starts,min,90s,goals,assists,goals_assists,non_penalty_goals,penalty_kicks,penalty_kicks_attempted,yellow_cards,red_cards,xg,non_penalty_xg,xag,non_pen_xg_plus_xag,progressive_carries,progressive_passes,progressive_passes_received,goals_per_90,assists_per_90,goals_assists_per_90,non_penalty_goals_per_90,non_penalty_goals_assists_per_90,xg_per_90,xag_per_90,xg_plus_xag_per_90,non_penalty_xg_per_90,non_penalty_xg_plus_xag_per_90,general_pos,age_range_stats,country_stats,continent_stats,player_id_stats
3166,Robert Lewandowski,Bayern Munich,28,303846,15800000,2017,25-29,Europe,1357.0,Centre-Forward,Poland,185.0,right,Borussia Dortmund,0.0,90.0,bundesliga,2014.0,1418,POL,FW,Bundesliga,28.0,1988.0,30,24,2172,24.1,29,2,31,23,6,7,1,0,27.9,22.4,2.4,24.8,44.0,71.0,220.0,1.20,0.08,1.28,0.95,1.04,1.16,0.10,1.26,0.93,1.03,Forward,25-29,Poland,Europe,1357
3495,Robert Lewandowski,Bayern Munich,29,306654,15946000,2018,25-29,Europe,1357.0,Centre-Forward,Poland,185.0,right,Borussia Dortmund,0.0,65.0,bundesliga,2014.0,1395,POL,FW,Bundesliga,29.0,1988.0,33,33,2957,32.9,22,7,29,19,3,4,2,0,30.7,27.5,8.9,36.4,79.0,92.0,241.0,0.67,0.21,0.88,0.58,0.79,0.93,0.27,1.21,0.84,1.11,Forward,25-29,Poland,Europe,1357
3853,Robert Lewandowski,Bayern Munich,30,307692,16000000,2019,30-34,Europe,1357.0,Centre-Forward,Poland,185.0,right,Borussia Dortmund,0.0,56.0,bundesliga,2014.0,1409,POL,FW,Bundesliga,30.0,1988.0,31,31,2759,30.7,34,4,38,29,5,5,5,0,30.7,26.8,6.6,33.5,70.0,91.0,245.0,1.11,0.13,1.24,0.95,1.08,1.00,0.22,1.22,0.87,1.09,Forward,30-34,Poland,Europe,1357
4234,Robert Lewandowski,Bayern Munich,31,442308,23000000,2020,30-34,Europe,1357.0,Centre-Forward,Poland,185.0,right,Borussia Dortmund,0.0,60.0,bundesliga,2014.0,1458,POL,FW,Bundesliga,31.0,1988.0,29,28,2458,27.3,41,7,48,33,8,9,4,0,31.3,24.2,4.7,28.8,51.0,67.0,185.0,1.50,0.26,1.76,1.21,1.46,1.14,0.17,1.31,0.88,1.06,Forward,30-34,Poland,Europe,1357
4621,Robert Lewandowski,Bayern Munich,32,442308,23000000,2021,30-34,Europe,1357.0,Centre-Forward,Poland,185.0,right,Borussia Dortmund,0.0,45.0,bundesliga,2014.0,1508,POL,FW,Bundesliga,32.0,1988.0,34,34,2946,32.7,35,3,38,30,5,5,2,0,33.2,29.3,4.4,33.7,46.0,81.0,268.0,1.07,0.09,1.16,0.92,1.01,1.01,0.13,1.15,0.90,1.03,Forward,30-34,Poland,Europe,1357


The join is now completed - we can now create a .py file to complete the full joining process. After this is done, we can create a seperate notebook that deals with joining the more advanced datasets together.